### TP 1 : General Observation on Order Book Data

###### Group : Alexandre BOISTARD, William ROCHE, Ethan TRENTIN

In [17]:
# Step 0 : Download of libraries
import numpy 
import scipy
import pandas as pd 
import matplotlib.pyplot as plt
import powerlaw
import statsmodels
import tqdm   
# import hawkes as hwk
import sys
import os

Question 1 : General statistical analysis on the dataset

In [15]:
# step 1 : load the data
Path = sys.path[0] + "\\Data\\SG"
# Let's loop on the file names inside the SG folder
for filename in os.listdir(Path):
    # Create a df name that changes along with the filename
    
    filename_df = pd.read_csv(Path + "\\" + filename, compression = 'gzip') 
    filename_df.drop(first_df.columns[0], axis = 1, inplace = True)

,ets,etype,eprice,eqty,eside,bp0,bq0,ap0,aq0
0,20170117:09:01:00.270164,A,45610,1400,B,46010,1066,46085,1445
1,20170117:09:01:00.312121,A,46485,700,S,46010,1066,46085,1445
2,20170117:09:01:00.358162,A,46000,124,B,46010,1066,46085,1445
3,20170117:09:01:00.359972,A,45950,182,B,46010,1066,46085,1445
4,20170117:09:01:00.360001,A,45970,22,B,46010,1066,46085,1445
...,...,...,...,...,...,...,...,...,...
841144,20170117:17:29:59.997678,A,45845,187,S,45820,377,45845,187
841145,20170117:17:29:59.997691,C,45720,1250,B,45820,377,45845,187
841146,20170117:17:29:59.997884,C,45765,110,B,45820,377,45845,187
841147,20170117:17:29:59.998327,A,45740,110,B,45820,377,45845,187
